In [8]:
import pandas as pd
import numpy as np
import re

In [9]:
pdb_file = open('PDB/2vz6_protein_chainA.pdb').readlines()

pdb = {'Record name': [],
       'Atom number': [],
       'Atom name': [],
       'AltLoc': [],
       'ResName': [],
       'ChainID': [],
       'ResSeq': [],
       'AChar': [],
       'X': [],
       'Y': [],
       'Z': [],
       'Occupancy': [],
       'Temperature factor': [],
       'Segment identifier': [],
       'Element symbol': [],
       'Charge': []
       }

for line in pdb_file:
    if re.match('^ATOM', line):
        pdb['Record name'].append(line[0:5])
        pdb['Atom number'].append(line[5:11])
        pdb['Atom name'].append(line[12:16])
        pdb['AltLoc'].append(line[16])
        pdb['ResName'].append(line[17:20])
        pdb['ChainID'].append(line[21])
        pdb['ResSeq'].append(line[22:26])
        pdb['AChar'].append(line[26])
        pdb['X'].append(line[30:38])
        pdb['Y'].append(line[38:46])
        pdb['Z'].append(line[46:54])
        pdb['Occupancy'].append(line[54:60])
        pdb['Temperature factor'].append(line[60:66])
        pdb['Segment identifier'].append(line[72:76])
        pdb['Element symbol'].append(line[76:78])
        pdb['Charge'].append(line[78:80])

pdb_df = pd.DataFrame(pdb,
                      columns=['Record name', 'Atom number', 'Atom name', 'AltLoc', 'ResName', 'ChainID', 'ResSeq', 'X', 'Y',
                               'Z', 'Occupancy', 'Temperature factor', 'Segment identifier', 'Element symbol', 'Charge']).set_index('Atom number')

pdb_df[['X', 'Y', 'Z', ]] = pdb_df[['X', 'Y', 'Z', ]].apply(pd.to_numeric)
pdb_df['ResSeq'] = pdb_df['ResSeq'].apply(pd.to_numeric)
pdb_df.head(5)

,Record name,Atom name,AltLoc,ResName,ChainID,ResSeq,X,Y,Z,Occupancy,Temperature factor,Segment identifier,Element symbol,Charge
Atom number,,,,,,,,,,,,,,
1,ATOM,N,,LEU,A,-5,72.154,72.898,24.108,1.00,54.74,,N,\n
2,ATOM,CA,,LEU,A,-5,73.207,72.316,24.986,1.00,56.20,,C,\n
3,ATOM,C,,LEU,A,-5,72.650,71.211,25.899,1.00,57.43,,C,\n
4,ATOM,O,,LEU,A,-5,73.208,70.110,25.966,1.00,57.73,,O,\n
5,ATOM,CB,,LEU,A,-5,73.859,73.417,25.835,1.00,55.48,,C,\n


In [10]:
x_size = pdb_df.X.max() - pdb_df.X.min()
y_size = pdb_df.Y.max() - pdb_df.Y.min()
z_size = pdb_df.Z.max() - pdb_df.Z.min()

print('Globular Size\n',
      '--size_x', x_size, '--size_y', y_size, '--size_z', z_size)

print('Molecular Center\n',
      '--center_x', pdb_df.X.max() - x_size / 2,
      '--center_y', pdb_df.Y.max() - y_size / 2,
      '--center_z', pdb_df.Z.max() - z_size / 2)

print('Volume\n', np.around(x_size * y_size * z_size, 1))

Globular Size
 --size_x 57.884 --size_y 50.181000000000004 --size_z 43.852999999999994
Molecular Center
 --center_x 45.214999999999996 --center_y 64.2885 --center_z 16.7755
Volume
 127378.8


In [13]:
residue = 234

x_c = np.around(pdb_df[pdb_df.ResSeq == residue].X.median(), 3)
y_c = np.around(pdb_df[pdb_df.ResSeq == residue].Y.median(), 3)
z_c = np.around(pdb_df[pdb_df.ResSeq == residue].Z.median(), 3)

print('Active Site Center')
print('--center_x', x_c, '--center_y', y_c, '--center_z', z_c)

Active Site Center
--center_x 20.678 --center_y 57.054 --center_z 0.898


In [12]:
pdb_df[pdb_df.ResSeq == 234]

,Record name,Atom name,AltLoc,ResName,ChainID,ResSeq,X,Y,Z,Occupancy,Temperature factor,Segment identifier,Element symbol,Charge
Atom number,,,,,,,,,,,,,,
1802,ATOM,N,,SER,A,234,20.717,58.485,1.604,1.00,19.21,,N,\n
1803,ATOM,CA,,SER,A,234,20.639,57.105,1.155,1.00,18.97,,C,\n
1804,ATOM,C,,SER,A,234,21.578,57.002,-0.018,1.00,19.08,,C,\n
1805,ATOM,O,,SER,A,234,21.721,57.978,-0.733,1.00,19.54,,O,\n
1806,ATOM,CB,,SER,A,234,19.233,56.760,0.641,1.00,20.00,,C,\n
1807,ATOM,OG,,SER,A,234,18.314,56.920,1.667,1.00,20.99,,O,\n


In [7]:
import os
import numpy as np
import pandas as pd
import tempfile
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc
from deepchem.utils import download_url, load_from_disk
import mdtraj as md
import nglview
from deepchem.utils.vina_utils import prepare_inputs
import deepchem as dc
from openmm.app import PDBFile
from openmm.app.internal.pdbstructure import PdbStructure
from openmm.app.internal.pdbx.reader.PdbxReader import PdbxReader
from openmm.app.element import hydrogen, oxygen
from openmm.app.forcefield import NonbondedGenerator
from openmm.app import *
from openmm.app import simtk
from openmm.app.internal import compiled
from IPython.display import display, Image
from pdbfixer import PDBFixer

ImportError: cannot import name 'simtk' from 'openmm.app' (/home/apolasa/.conda/envs/PDBFixer/lib/python3.9/site-packages/openmm/app/__init__.py)

In [2]:
protein_mdtraj = md.load_pdb('PDB/2vz6_protein_chainA.pdb')
ligand_mdtraj = md.load_pdb('PDB/Target1.pdb')

In [3]:
finder = dc.dock.binding_pocket.ConvexHullPocketFinder()
pockets = finder.find_pockets('PDB/2vz6_protein_chainA.pdb')
len(pockets)  # number of identified pockets

42

In [4]:
vpg = dc.dock.pose_generation.VinaPoseGenerator()

In [5]:
!mkdir -p vina_test

In [6]:
%%time
complexes, scores = vpg.generate_poses(molecular_complex=('PDB/2vz6_protein_chainA.pdb', 'PDB/Target1.pdb'),  # protein-ligand files for docking,
                                       out_dir='vina_test',
                                       generate_scores=True
                                      )

ImportError: This function requires pdbfixer